In [1]:
from pyspark.sql import SparkSession
from delta.tables import *
from delta import *

In [2]:
 spark = SparkSession.builder \
        .appName("ProcessJSONToDelta") \
        .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.3.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

25/02/06 15:08:29 WARN Utils: Your hostname, NB-WXET1590 resolves to a loopback address: 127.0.1.1; using 172.30.88.241 instead (on interface eth0)
25/02/06 15:08:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/avyas/miniconda3/envs/testairbyte/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/avyas/.ivy2/cache
The jars for the packages stored in: /home/avyas/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-62455a93-ab8f-42f5-aba6-5602e254fb69;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 540ms :: artifacts dl 29ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [3]:
delta_table = DeltaTable.forPath(spark, '../data/delta_table/core_data/')

In [4]:
schema = delta_table.toDF().schema
schema

StructType([StructField('abstract', StringType(), True), StructField('authors', ArrayType(StringType(), True), True), StructField('contributors', ArrayType(StringType(), True), True), StructField('coreId', StringType(), True), StructField('datePublished', StringType(), True), StructField('doi', StringType(), True), StructField('downloadUrl', StringType(), True), StructField('enrichments', StructType([StructField('documentType', StructType([StructField('type', DoubleType(), True)]), True), StructField('references', ArrayType(StructType([StructField('authors', ArrayType(StringType(), True), True), StructField('cites', StringType(), True), StructField('date', StringType(), True), StructField('doi', StringType(), True), StructField('id', LongType(), True), StructField('raw', StringType(), True), StructField('title', StringType(), True)]), True), True)]), True), StructField('fullText', StringType(), True), StructField('fullTextIdentifier', StringType(), True), StructField('identifiers', Arr

In [ ]:
metadata = delta_table.history()
metadata.show()

In [5]:
delta_table.toDF().show(2)

25/02/06 15:08:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 9:============================================>              (3 + 1) / 4]

+--------+--------------------+------------+------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+
|abstract|             authors|contributors|coreId|datePublished|                 doi|         downloadUrl|         enrichments|            fullText|  fullTextIdentifier|         identifiers|journals|        language|                 oai|        pdfHashValue|        publisher|        rawRecordXml|           relations|             subject|               title|              topics|year|
+--------+--------------------+------------+------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+----------------+-------

In [7]:
spark.sql ("""CREATE TABLE delta_table
                USING delta
                LOCATION '/home/avyas/projects/project_os/elt_articles//data/delta_table/core_data/'""")

DataFrame[]

In [ ]:
# spark.sql("DROP TABLE delta_table1")

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.sql("SELECT current_database()").show()

In [ ]:
spark.sql("""DESCRIBE delta_table""").show()

In [ ]:
spark.sql("""Select COUNT(*) from delta_table """).show()

In [8]:
spark.sql("""SELECT doi, COUNT(*) as count
            FROM delta_table
            GROUP BY doi
            HAVING COUNT(*) > 1;
            """).show()

+--------------------+-----+
|                 doi|count|
+--------------------+-----+
|      10.1086/426597|    2|
|10.5194/angeo-26-...|    2|
|    10.1002/rcm.4528|    2|
|10.1103/PhysRevC....|    2|
|10.1108/EUM000000...|    2|
|        10.1112/blms|    3|
|        10.1155/2007|    2|
|10.1057/palgrave....|    2|
|10.1103/PhysRevE....|    2|
|   10.1144/1470-9236|   20|
|   10.1063/1.2126117|    2|
|10.1093/bioinform...|    3|
|   10.1137/050648717|    2|
|10.1017/S01433857...|    2|
|        10.1093/iclq|   14|
|10.1142/S02182025...|    2|
|10.1017/S17458552...|    2|
|         10.1093/aob|    2|
|          10.1093/gh|    3|
|      10.1093/indlaw|   13|
+--------------------+-----+
only showing top 20 rows

